# 1. Threading in PyQT

## 1.1 Why we need multi threading in Qt application?

A common problem when building Python GUI applications is `locking up/freeze` of the interface when attempting to perform long-running background tasks. This happens because Qt application are **event based**, which means execution is driven in response to user interaction, signals and timers. For example, clicking on a button creates an event which your application subsequently handles to produce some expected output. Events are pushed onto and taken off an `event queue` and processed sequentially.

You should see below code on all Qt applications which launches your Qt application
```python
# create a main thread for your qt application
app = QApplication(sys.argv)
# create your custom qt GUI
window = MainWindow()
# starts the main thread, which runs the event loop of the app and your custom GUI
# the main thread also handles all window communication with the host OS.
sys.exit(app.exec())
```

By default, all runs in the main thread synchronously. It means if an execution triggered by the event loop, all other things will be `blocked` such as window communication and GUI interaction.

If what you're doing is simple, and returns control to the GUI loop quickly, this freeze will be imperceptible to the user. However, if you need to perform `longer-running tasks`, for example opening/writing a large file, downloading some data, or rendering some complex image, there are going to be problems. Your application will appear to be `unresponsive` (because it is) until the task terminates. Because your app is no longer communicating with the OS.


**The solution is**: get your work out of the `main thread` (and into another thread). PyQt (via Qt) provides an straightforward interface(**QThread**) to do exactly that.

## 1.2 A freezing example

In below example, the `Counting label` reflects the total number of clicks on the `Click me! button`. Clicking the `Long-Running Task! button` will launch a task that takes a lot of time to finish. Your long-running task could be a file download, a query to a large database, or any other resource-intensive operation.

In [ ]:
import sys
from time import sleep

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (
    QApplication,
    QLabel,
    QMainWindow,
    QPushButton,
    QVBoxLayout,
    QWidget,
)


class Window(QMainWindow):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.clicksCount = 0
        self.setupUi()

    def setupUi(self):
        self.setWindowTitle("Freezing GUI")
        self.resize(300, 150)
        self.centralWidget = QWidget()
        self.setCentralWidget(self.centralWidget)
        # Create and connect widgets
        self.clicksLabel = QLabel("Counting: 0 clicks", self)
        self.clicksLabel.setAlignment(Qt.AlignmentFlag.AlignHCenter | Qt.AlignmentFlag.AlignVCenter)
        self.stepLabel = QLabel("Long-Running Step: 0")
        self.stepLabel.setAlignment(Qt.AlignmentFlag.AlignHCenter | Qt.AlignmentFlag.AlignVCenter)
        self.countBtn = QPushButton("Click me!", self)
        self.countBtn.clicked.connect(self.countClicks)
        self.longRunningBtn = QPushButton("Long-Running Task!", self)
        self.longRunningBtn.clicked.connect(self.runLongTask)
        # Set the layout
        layout = QVBoxLayout()
        layout.addWidget(self.clicksLabel)
        layout.addWidget(self.countBtn)
        layout.addStretch()
        layout.addWidget(self.stepLabel)
        layout.addWidget(self.longRunningBtn)
        self.centralWidget.setLayout(layout)

    def countClicks(self):
        self.clicksCount += 1
        self.clicksLabel.setText(f"Counting: {self.clicksCount} clicks")

    def reportProgress(self, n):
        self.stepLabel.setText(f"Long-Running Step: {n}")

    def runLongTask(self):
        """Long-running task in 5 steps. Each step takes 1 sec to finish """
        for i in range(5):
            sleep(1)
            self.reportProgress(i + 1)


app = QApplication(sys.argv)
win = Window()
win.show()
sys.exit(app.exec())


After running the above application, you can notice if you click the `Long-Running Task! button`, then the application becomes frozen and unresponsive. The buttons no longer respond to clicks and the labels don’t reflect the application’s state.

After five seconds, the application’s GUI gets updated again. The Counting label shows all the clicks during the frozen time, reflecting the clicks that occurred while the GUI was frozen. The `Long-Running Step label doesn’t reflect the progress of your long-running operation`. It jumps from zero to five without showing the intermediate steps.

> Even though your application’s GUI freezes during the long-running task, the application still registers events such as clicks and keystrokes. It’s just unable to process them until the main thread gets released.

## 1.3 Introduction of Multithreading

You can skip this section, if you are familiar with Multithreading.

### Process and Thread

A **thread** is a separate flow of execution. In most operating systems, `a thread is a component of a process`, and `processes` can have multiple threads executing `concurrently`. Each **process** represents an instance of a program or application that is currently running in a given computer system.

You can have as many threads as you need. The challenge is `determining the right number of threads` to use. If you’re working with `I/O-bound threads`, then the number of threads will be limited by your available system resources. On the other hand, if you’re working with `CPU-bound threads`, then you’ll benefit from having a number of threads that is equal to or less than the number of `CPU cores` in your system.


### Multithreading

Building programs that are capable of running multiple tasks using different threads is a programming technique known as **multithreaded programming**. Ideally, with this technique, several `tasks run independently at the same time`. However, this isn’t always possible. There are at least two elements that can prevent a program from running several threads in parallel:

- The central processing unit (CPU)
- The programming language

For example, if you have a `single-core CPU` machine, then you can’t run multiple threads at the same time. If you have a `multi-core CPU` machine or a computer cluster, then you might be able to run multiple threads at the same time. In this case, your programming language becomes an important factor.

Some programming languages have internal components that actually prohibit the real execution of multiple threads in parallel. In these cases, threads just appear to run in parallel (but they are not in reality) because they take advantage of the task scheduling system.

Multithreaded programs are usually more difficult to write, maintain, and debug than single-threaded programs because of the complexity related to sharing resources between threads, synchronizing data access, and coordinating thread execution. This can cause several problems:
- **Race condition**: happens when the application’s behavior becomes `nondeterministic due to the unpredictable order of events`. It’s often the result of two or more threads accessing a shared resource without proper synchronization. For example, reading and writing memory from different threads can lead to a race condition if the reading and writing operations are performed in the wrong order.

- **Deadlock**: happens when threads wait indefinitely for a locked resource to be freed. For example, if a thread locks a resource and doesn’t unlock it after use, then other threads won’t be able to use that resource and will wait indefinitely. Deadlocks can also happen if thread A is waiting for thread B to unlock a resource, and thread B is waiting for thread A to unlock a different resource. Both threads will end up waiting forever.

- **Livelock**: happens when two or more threads repeatedly act in response to each other’s actions. Livelocked threads are unable to make further progress on their specific task because they’re too busy responding to each other. However, they’re not blocked or dead.

- **Starvation**: happens when a process never gains access to the resources it needs for finishing its work. For example, if you have a process that can’t gain CPU time access, then the process is starving for CPU time and can’t do its work.

> When building multithreaded applications, you need to be careful to protect your resources from concurrent writing or state modification access. In other words, you need to prevent multiple threads from accessing a given resource at the same time.

A wide range of applications can benefit from using multithreaded programming in at least three ways:

- Making your applications faster by taking advantage of multi-core processors
- Simplifying the application structure by dividing it into smaller subtasks
- Keeping your application responsive and up to date by offloading long-running tasks to worker threads

> In Python’s C implementation, also known as CPython, threads don’t run in parallel. CPython has a global interpreter lock (GIL), which is a lock that basically allows only one Python thread to run at a time.

This can negatively affect the performance of threaded Python applications because of the overhead that results from the context switching between threads. However, multithreading in Python can help you solve the problem of freezing or unresponsive applications while processing long-running tasks.

## 1.4 Multithreading in PyQt With QThread

**PyQt (Qt)** provides its own infrastructure to create multithreaded applications using **QThread**. PyQt applications can have two different kinds of threads:

- Main thread
- Worker threads

### The main Thread

In PyQt applications, `the main thread of execution is also known as the GUI thread` because it handles all widgets and other GUI components. Python starts this thread when you run the application. The application’s event loop runs in this thread after you call .exec() on the QApplication object. This thread handles your windows, dialogs, and also the communication with the host operating system.


It’s important to note that **you must create and update all your widgets in the main/GUI thread**. However, you can execute other long-running tasks in worker threads and use their results to feed the GUI components of your application. This means that GUI components will act as consumers that are fed information from the threads performing the actual work.

### The worker threads

You can create as many worker threads as you need in your PyQt applications. Worker threads are secondary threads of execution that you can use to offload long-running tasks from the main thread and prevent GUI freezing.

You can create worker threads using `QThread`. `Each worker thread can have its own event loop and support PyQt’s signals and slots mechanism to communicate with the main thread. If you create an object from any class that inherits from QObject in a particular thread, then that object is said to belong to, or have an affinity to, that thread. Its children must also belong to the same thread.`

QThread isn’t a thread itself. It’s a wrapper around an operating system thread. The real thread object is created when you call QThread.start().

QThread provides a high-level application programming interface (API) to manage threads. This API includes signals, such as .started() and .finished(), that are emitted when the thread starts and finishes. It also includes methods and slots, such as .start(), .wait(), .exit(), .quit(), .isFinished(), and .isRunning().

> Like with any other threading solutions, with QThread you must protect your data and resources from concurrent, or simultaneous, access. Otherwise you’ll face a lot of problems, including deadlocks, data corruption, and so on.


## 1.5 Using QThread vs Python’s threading

Python standard library already offers a [threading module](https://docs.python.org/3/library/threading.html#module-threading). You might be wondering what should I use in my PyQt applications, Python’s native thread or PyQt’s thread? The answer is that it depends.

For example, if you’re building a GUI application that will also have a web version, then Python’s threads might make more sense because your back end won’t depend on PyQt at all. However, if you’re building bare PyQt applications, then PyQt’s threads are for you.

Using PyQt’s thread support provides the following benefits:

- **Thread-related classes**: are fully integrated with the rest of PyQt infrastructure.
- **Worker threads** can have their own event loop, which enables event handling.
- **Interthread communication** is possible using signals and slots.

A rule of thumb might be to use PyQt’s thread support if you’re going to interact with the rest of the library, and use Python’s thread support otherwise.

## 1.6 Using QThread to prevent freezing GUIs

In this section, we will use **QThread** to create and manage a worker thread to offload the `long-running task` from the main thread, so the GUI will not freeze anymore.

According to [Qt’s documentation](https://doc.qt.io/qt-6.2/qthread.html#details), there are two main ways to create worker threads with QThread:

1. Instantiate QThread directly and create a worker QObject, then call .moveToThread() on the worker using the thread as an argument. The worker must contain all the required functionality to execute a specific task.
2. Subclass QThread and reimplement .run(). The implementation of .run() must contain all the required functionality to execute a specific task.

Instantiating QThread provides a parallel event loop. An event loop allows objects owned by the thread to receive signals on their slots, and these slots will be executed within the thread.

On the other hand, subclassing QThread allows running parallel code without an event loop. With this approach, you can always create an event loop by calling `exec()` explicilty.

In below example, we will use the first approach, which requires the following steps:
1. Prepare a worker object by subclassing QObject and put your long-running task in it.
2. Create a new instance of the worker class.
3. Create a new QThread instance.
4. Move the worker object into the newly created thread by calling .moveToThread(thread).
5. Connect the required signals and slots to guarantee interthread communication.
6. Call .start() on the QThread object.

Let's retake the above freezing example, we did follow update
1. Add a Worker class
2. rewrite the `runLongTask` method of the main window

In [ ]:
import sys
from time import sleep

from PyQt6.QtCore import Qt, QObject, pyqtSignal, QThread
from PyQt6.QtWidgets import (
    QApplication,
    QLabel,
    QMainWindow,
    QPushButton,
    QVBoxLayout,
    QWidget,
)


# Step 1: Create a worker class
class Worker(QObject):
    """
    In this class, we create two signals, finished and progress. The signals must be class attributes
    """
    finished: pyqtSignal = pyqtSignal()
    progress = pyqtSignal(int)

    def run(self):
        """Long-running task."""
        for i in range(5):
            sleep(1)
            self.progress.emit(i + 1)
        self.finished.emit()


class Window(QMainWindow):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.clicksCount = 0
        self.setupUi()

    def setupUi(self):
        self.setWindowTitle("Freezing GUI")
        self.resize(300, 150)
        self.centralWidget = QWidget()
        self.setCentralWidget(self.centralWidget)
        # Create and connect widgets
        self.clicksLabel = QLabel("Counting: 0 clicks", self)
        self.clicksLabel.setAlignment(Qt.AlignmentFlag.AlignHCenter | Qt.AlignmentFlag.AlignVCenter)
        self.stepLabel = QLabel("Long-Running Step: 0")
        self.stepLabel.setAlignment(Qt.AlignmentFlag.AlignHCenter | Qt.AlignmentFlag.AlignVCenter)
        self.countBtn = QPushButton("Click me!", self)
        self.countBtn.clicked.connect(self.countClicks)
        self.longRunningBtn = QPushButton("Long-Running Task!", self)
        self.longRunningBtn.clicked.connect(self.runLongTask)
        # Set the layout
        layout = QVBoxLayout()
        layout.addWidget(self.clicksLabel)
        layout.addWidget(self.countBtn)
        layout.addStretch()
        layout.addWidget(self.stepLabel)
        layout.addWidget(self.longRunningBtn)
        self.centralWidget.setLayout(layout)

    def countClicks(self):
        self.clicksCount += 1
        self.clicksLabel.setText(f"Counting: {self.clicksCount} clicks")

    def reportProgress(self, n):
        self.stepLabel.setText(f"Long-Running Step: {n}")

    def runLongTask(self):
        # Step 2: Create a QThread object
        self.thread = QThread()
        # Step 3: Create a worker object
        self.worker = Worker()
        # Step 4: Move worker to the thread
        self.worker.moveToThread(self.thread)
        # Step 5: Connect signals and slots
        self.thread.started.connect(self.worker.run)
        self.worker.finished.connect(self.thread.quit)
        self.worker.finished.connect(self.worker.deleteLater)
        self.thread.finished.connect(self.thread.deleteLater)
        self.worker.progress.connect(self.reportProgress)
        # Step 6: Start the thread
        self.thread.start()

        # Final resets
        self.longRunningBtn.setEnabled(False)
        self.thread.finished.connect(
            lambda: self.longRunningBtn.setEnabled(True)
        )
        self.thread.finished.connect(
            lambda: self.stepLabel.setText("Long-Running Step: 0")
        )


app = QApplication(sys.argv)
win = Window()
win.show()
sys.exit(app.exec())